**Minería de Datos**

*CC5205-1 - Otoño 2022*

Integrantes:
- Felipe Urrutia
- Camilo Carvajal
- Gianluca Musso
- Jose Ignacio Saffie
- Matias Lopez Roman

## Introducción

Durante los últimos años hemos vivido un aumento considerable tanto en la cantidad y acceso a grandes cantidades de datos, como también a un poder de cálculo que ha proliferado las posibilidades para procesar estos.
En este contexto, el procesamiento de lenguaje natural ha adquirido un rol protagónico. A los métodos estadísticos y probabilísticos se le ha agregado modelos de lenguaje con impresionantes capacidades para procesar, clasificar en incluso generar texto. 

Una de sus tareas más populares ha sido el análisis de sentimiento, que trata de predecir el valor emocional de algún documento. Esta es una tarea difícil incluso para seres humanos, pero muy útil puesto a que sus aplicaciones son variadas. Por ejemplo, es de interés para  una empresa saber el sentimiento de los clientes hacia sus productos. Esto se puede lograr procesando grandes cantidades de tweets o de reseñas. 
En su versión más explorada, el análisis de sentimiento trata de predecir cuan positivo o negativo es un mensaje, sin embargo este proceso puede ser refinado aún más. La complejidad del lenguaje humano nos sugiere que podemos considerar modelar el valor sentimental de un mensaje en un muchas más dimensiones.

El dataset **Multilingual Emoji Prediction** (Barbieri et al. 2010, descargable [con este link](https://github.com/fvancesco/Semeval2018-Task2-Emoji-Detection/blob/master/dataset/Semeval2018-Task2-EmojiPrediction.zip?raw=true)) contiene alrededor de 500k tweets, todos conteniendo un emoji, de un conjunto de 20 comúnmente usados. El desafío de base es predecir el emoji en cuestión desde el texto del tweet. Esta tarea puede ser interpretada como una de análisis de sentimiento multimodal puesto a que el emoji comúnmente denota información no verbal del mensaje o contexto, muchas veces emocional.

Barbieri, F., Camacho-Collados, J., Ronzano, F., Espinosa Anke, L., Ballesteros, M., Basile, V., ... & Saggion, H. (2018). Semeval 2018 task 2: Multilingual emoji prediction. In 12th International Workshop on Semantic Evaluation (SemEval 2018) (pp. 24-33). Association for Computational Linguistics. [http://dx.doi.org/10.18653/v1/S18-1003](http://dx.doi.org/10.18653/v1/S18-1003)

Librerias

In [1]:
import pandas as pd
import numpy as np
# pd.set_option("max_rows", None)
import pickle
from string import punctuation
import re

from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
# from astropy.visualization import hist
sns.set_theme(style="whitegrid")
plt.rc('axes', titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rcParams.update({'font.size': 16})
plt.rcParams['axes.titlesize'] = 16
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams.update({'lines.markeredgewidth': 1})
plt.rcParams.update({'errorbar.capsize': 2})
import random

Rutas

In [2]:
roots = {
    "main": "/content/drive/MyDrive/DM22",
    "main/data": "/content/drive/MyDrive/DM22/Data"
}

In [3]:
file_names = {
    "mapping/es": "/mapping/es_mapping.txt",
    "mapping/us": "/mapping/us_mapping.txt",

    "test/es/labels": "/test/es_test.labels",
    "test/es/text": "/test/es_test.text",

    "test/us/labels": "/test/us_test.labels",
    "test/us/text": "/test/us_test.text",

    "trial/es/labels": "/trial/es_trial.labels",
    "trial/es/text": "/trial/es_trial.text",

    "trial/us/labels": "/trial/us_trial.labels",
    "trial/us/text": "/trial/us_trial.text"
}

```
Data
|-mapping
|--es_mapping.txt
|--us_mapping.txt
|-test
|--es_test.labels
|--es_test.text
|--us_test.labels
|--us_test.text
|-trial
|--es_trial.labels
|--es_trial.text
|--us_trial.labels
|--us_trial.text
```

Leer mapping

In [4]:
with open(roots["main/data"]+file_names["mapping/es"]) as f:
  dm = f.read()

In [5]:
df_mapping = pd.DataFrame([l.strip().split("\t") for l in dm.split("\n")], columns="LABEL EMJOI NAME".split())
df_mapping["LABEL"] = df_mapping["LABEL"].apply(int)
df_mapping

,LABEL,EMJOI,NAME
0,0,❤,_red_heart_
1,1,😍,_smiling_face_with_hearteyes_
2,2,😂,_face_with_tears_of_joy_
3,3,💕,_two_hearts_
4,4,😊,_smiling_face_with_smiling_eyes_
5,5,😘,_face_blowing_a_kiss_
6,6,💪,_flexed_biceps_
7,7,😉,_winking_face_
8,8,👌,_OK_hand_
9,9,🇪🇸,_Spain_


Leer tweets & label

In [6]:
with open(roots["main/data"]+file_names["trial/es/text"]) as f:
  dt = f.read().strip().split("\n")

with open(roots["main/data"]+file_names["trial/es/labels"]) as f:
  dl = f.read().strip().split("\n")

In [7]:
len(dt), len(dl)

(10000, 10000)

In [8]:
df_data = pd.DataFrame([{"TWEET": dt[i], "LABEL": dl[i]} for i in range(1000)])
df_data["LABEL"] = df_data["LABEL"].apply(int)
df_data

,TWEET,LABEL
0,"Plaza de Oriente , Madrid .......#madrid #city...",9
1,"Por ser la columna de mi templo, por ser lo me...",0
2,Me gustan las motos! #cheste2016 #nicoabad #el...,2
3,"Sevilla tiene un color especial, Sevilla tiene...",16
4,Que (la) Chipi no se caiga .Cuánto os quiero c...,1
...,...,...
995,@ Torrejón de Velasco,0
996,"""@OliverTuits: Metáfora de lo que le va a pasa...",2
997,No todos los días se cumplen años. Esta vez po...,0
998,#Teniendo los #MejoresRecuerdos para que pedir...,13


In [9]:
df = df_data.merge(df_mapping, how="left", on="LABEL")
df

,TWEET,LABEL,EMJOI,NAME
0,"Plaza de Oriente , Madrid .......#madrid #city...",9,🇪🇸,_Spain_
1,"Por ser la columna de mi templo, por ser lo me...",0,❤,_red_heart_
2,Me gustan las motos! #cheste2016 #nicoabad #el...,2,😂,_face_with_tears_of_joy_
3,"Sevilla tiene un color especial, Sevilla tiene...",16,🎶,_musical_notes_
4,Que (la) Chipi no se caiga .Cuánto os quiero c...,1,😍,_smiling_face_with_hearteyes_
...,...,...,...,...
995,@ Torrejón de Velasco,0,❤,_red_heart_
996,"""@OliverTuits: Metáfora de lo que le va a pasa...",2,😂,_face_with_tears_of_joy_
997,No todos los días se cumplen años. Esta vez po...,0,❤,_red_heart_
998,#Teniendo los #MejoresRecuerdos para que pedir...,13,😜,_winking_face_with_tongue_


In [10]:
df[df["LABEL"] == 0].head(30)

,TWEET,LABEL,EMJOI,NAME
1,"Por ser la columna de mi templo, por ser lo me...",0,❤,_red_heart_
12,Perfecto. Te quiero @ Pabellón Municipal De Ta...,0,❤,_red_heart_
13,Miss my flawless fabulososssssss @ Primark Gra...,0,❤,_red_heart_
20,Si no sabes explicarlo es porque merece la pen...,0,❤,_red_heart_
31,Muchisimas felicidaes mi amor pd no me mates p...,0,❤,_red_heart_
33,@ Parque Federico García Lorca,0,❤,_red_heart_
36,Orgulloso de ti! miriam3694 #gym #nutrition #b...,0,❤,_red_heart_
40,Por los reencuentros bonitos @user te quiero @...,0,❤,_red_heart_
41,Feliz día Papá. @ Mercado San Anton,0,❤,_red_heart_
60,Remates en terciopelo #rojo para las pamelas d...,0,❤,_red_heart_


In [11]:
pd.DataFrame(df["LABEL"].value_counts()).reset_index().rename(columns={"LABEL": "FREC.", "index":"LABEL"}).merge(df_mapping, how="left", on="LABEL")

,LABEL,FREC.,EMJOI,NAME
0,0,196,❤,_red_heart_
1,1,133,😍,_smiling_face_with_hearteyes_
2,2,98,😂,_face_with_tears_of_joy_
3,3,75,💕,_two_hearts_
4,4,59,😊,_smiling_face_with_smiling_eyes_
5,6,44,💪,_flexed_biceps_
6,7,42,😉,_winking_face_
7,8,38,👌,_OK_hand_
8,9,36,🇪🇸,_Spain_
9,5,36,😘,_face_blowing_a_kiss_


### Ideas exploración

- medidas de tendencia central para la frecuencia de los emoji (con dispersión)
- medidas de tendencia central para tweet length
- cuantiles para lo anterior
- graficar la frecuencia de emojis en los 3 sets
- graficar cuantiles de longitud
- frecuencia de palabras, con cuantiles. pre y post stemming
- covarianza y/o tablas de contingencia, entre palabras y emojis